# 2. Keys and Blockchain Addresses

In the first session we learned about digital signatures, a key application of public key cryptography.  
We also encrypted messages to prevent third parties from viewing them; however, on Bitcoin and Ethereum blockchains data is usually unencrypted. Cryptography is used to ensure consistency and integrity of data.  
> Users of blockchains can protect their privacy by avoiding associating their identity to the data held on the blockchain.   
> Privacy on blockchains is not guaranteed - as it is with encrypting messages - it depends on many factors.   
> Blockchains ensure ***pseudonymity*** defined as the near-anonymous state in which a user has a consistent identifier that is not their real name: a pseudonym.      
> We will cover this topic in details in a teaching about enhancing Know Your Client ("KYC") processes through Know Your Crypto Assets ("KYCA") activities.  
>
To sign messages we created a key pair and distributed the public keys.  
💡 In this session we will use the keypair to interact with the two main blockchains, Ethereum and Bitcoin    
We will also focus on the risk of generating private keys and securing them.  
💭 A blockchain address is a SECP256K1 public key encoded in accordance to the recipe prescribed in the Bitcoin whitepaper and implemented by all bitcoin complienat software    
💭 The private key is a 64 characters / 256 bits / 32 bytes hexadecimal number (a number with a radix or base of 16) that complies with the SECP256K1 specifications.  
💡 We use the SECP256K1 because it is the standard algorithm mandated by Bitcoin and Ethereum.  
> The recipes to encode addresses (i.e. deriving a hash of some sort) from a SECP256K1 public key) are contained in the Bitcoin and Ethereum whitepapers and conveniently packged in libraries.  
> We can use the same private key to generate addresses for both Bitcoin and Ethereum blockchains   
>

**Bitcoin Addresses**
From the Public key (a 128 hex characters with a lenghth of  512 bits or 64 bytes) we can derive four different types of addresses.  
> ***Taproot address*** - P2TR or Bech32m, starts with ***bc1p*** offers multi-key addresses. It looks the same as single-key addresses giving users of multi-key increased privacy and enables more advanced scripting, enabling more complex smart contracts to be built on bitcoin. It is case insensitive.    
> ***SegWit address*** - P2WPKH,  A pay-to-witness-public-key-hash, starts with ***bc1q*** and is case insensitive.  A different encoding for the Pay-to-Witness-Script-Hash (P2WSH) gives a longer key (32 bytes as opposed to 20) with an identical prefix.     
> ***Script address*** - P2SH, starts with ***3***. A pay-to-script-hash (P2SH), or script address, can have additional rules and functionality attached to the address. Script addresses are commonly used for multi-sig addresses, which can specify that signatures from several keys are required to authorize the transaction. The address is case sensitive.  
> ***Legacy address*** - P2PKH, starts with ***1***. A pay-to-pubkey-hash (P2PKH), or legacy address, is the oldest and original bitcoin address format. The address is case sensitive.   

**Ethereum Addresses**
There are two types of accounts on the Ethereum network, externally owned accounts and contract accounts.   
> ***Externally owned accounts*** are derived from the a valid SECP256K1 public key, just like the addresses of the Bitcoin blockhcian (the recipe, though, is different)  
> ***Contract accounts*** are generated automatically by the Ethereum virtual machine when a smart contract is deployed, The do not have a corresponding private key and they are operated by those that can access it.  
>

Unlike Bitcoin accounts, Ethereum accounts have state fields:  
> Nonce: This represents the number of transactions initiated by the account.    
> Balance: This represents the balance of the account in Wei.  
> CodeHash: The hash of the smart contract code is stored in this field.  For externally owned accounts this field is empty.     
> Storage: This field contains the data of the storage variables within the smart contract. For externally owned accounts this field is empty.  
>

We will consider the differences between Ethereum and bitcoin in future sessions, for the moment:
> Bitcoin uses the unspent transaction output (UTXO) model, while Ethereum uses an account-based model. In the UTXO model, a transaction consists of inputs and outputs. The account model is more akin to the traditional accounting model.  
> The Ethereum virtual machine is a touring-complete system, whereas the Bitcoin virtual machine supports a scripting language with limited funcitonality.    
>

**The code**    
Below, we define the functions to generate a SECP256K1 public key from a given private key and encode it into Bitcoin and Ethereum addresses.

#Cell 1

In [ ]:
print("Generating functions...") 
%pip install base58 
%pip install segno
%pip install ecdsa -q
import ecdsa 
import binascii
import base58
import segno
import ecdsa
from Crypto.Hash import RIPEMD160
from Crypto.Hash import SHA256
from Crypto.Hash import keccak
  
def generate_key(s): # Note, this function now requires an input, your desired private key, represented as an integer number
    #The private key is an integer number of you choice
    #The function then converts it into an hexadecimal number
    #The function will also create a file with the qr code of your private key - saves upi from typing 60 odd zeros on your mobile!
    s = str(hex(s))[2:]
    s = s.zfill(64)
    private_key = bytes.fromhex(s)
    kpr = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
    private_key_hex = kpr.to_string().hex()
    kpu = kpr.get_verifying_key() 
    public_key_hex = kpu.to_string().hex()
    return public_key_hex, private_key_hex

def qr_code(private_key): #This function create a file  
    qrcode = segno.make_qr(private_key_hex)
    qrcode.save(public_key_hex + ".png", scale=10)
    return qrcode

def eth_account(public_key):  #The recipe to obtain an ethereum account from a public key is simple
    keccak_hash = keccak.new(digest_bits=256)
    keccak_hash.update(binascii.unhexlify(public_key))
    keccak_digest = keccak_hash.hexdigest()
    return "0x" + keccak_digest[-40:]

def btc_account(public_key_hex): #The recipe to obtain a bitcoin account from a public key is elaborate
    # I have numbered the steps from 1 to 9
    #1) Add prefix "04" to public ECDSA Key 
    step1 = '04' + public_key_hex
    #2) SHA-256 hash of 1
    step2 = SHA256.new(binascii.unhexlify(step1)).digest()
    #3) RIPEMD-160 Hash of 2
    step3 = RIPEMD160.new(step2).hexdigest()
    #4) Adding network bytes to 3; "00" is for main net
    step4 = bytes(bytearray.fromhex("00" + step3))
    step4a = "00" + step3
    #5) SHA-256 hash of 4
    step5 = SHA256.new(step4).digest()
    #6) SHA-256 hash of 5
    step6 = SHA256.new(step5).hexdigest()
    #7) First four bytes of 6
    step7 = step6[:8]
    #8) Adding 7 at the end of 4
    step8 = step4a+step7
    #9) Base58 encoding of 8
    step9 = (base58.b58encode(bytes.fromhex(step8)).decode())
    return step9


print("Functions generated, you can now use them on this workbook to derive Ethereum and Bitcoin addresses from a given private key.") 

#Cell 2

**Generate a private key**  
The private key, ultimately, is just a number. It has to satisfy the requirements of the SECP256K1 standard, which prescribes that the number be:
- greater than 0
- smaller than 115792089237316195423570985008687907852837564279074904382605163141518161494336.  

💡 This is a very large number that guarantees no "collisions" when generating new addresses.  
>A collision occurs when the same private key is generated by multiple algorithms or individuals.    
>What if I could test all possible private keys, from 1 to the maximum number allowed by SECP256K1?  
>Current computers would take several years to crack a given public key; quantum computers could take seconds.  

💭 You can import the private key on a wallet of your choice, for example metamask https://metamask.io/  
> You can use the function *paper_wallet* to generate a qr code of the private key.  

#Cell 3

**Exercise**  
Use the cell below to generate public keys and check the corresponent addresses on a blockchain explorer.  
Use the functions btc_account(***public_key_hex***) and eth_account(***public_key_hex***) where ***public_key_hex*** is a SECP256K1 public key (in hexadecimal format) generated from an integer number of your choicewith the generate_key  function.    
💭 You can use https://live.blockcypher.com or any other block explorer to check transactions and balances associated with your accounts.  

#Cell 4

In [ ]:
#your code:

#Cell 5

**Conclusions**  
This module allowed you to generate blockchain addresses to receive digital assets.    
You have learned about the perils of generating private keys using poor entropy.  
In cryptography, entropy refers to the randomness collected by a system for use in algorithms that require random seeds. A lack of good entropy can leave a crypto system vulnerable and unable to encrypt data securely.  
> The NIST Federal Information Processing Standard (FIPS) PUB 140-2 requires the use of a Deterministic Random Bit Generator ("DRBG"), compliant with SP 800-90A (Recommendation for Random Number Generation Using Deterministic Random Bit Generators) and an entropy source compliant with SP 800-90B (Recommendation for the Entropy Sources Used for Random Bit Generation).  
>

You should also focus on the risks of generating keys on untrusted infrastructure. For example:
> The computer and the server used may have vulnerabilities that an attacker could exploit to copy your private keys.  
> The software utilised to generate keys may have bugs (for example: code logic errors, Type confusion, random device errors, poorly handled exceptions, memory reference errors, memory corruption).  
> The software may be compromised.  



💡 Keys should be generated only on trusted systems that are not connected to the Internet.  
💡 Keys should be stored securely. A wallet like metamask may be suitable for individuals to handle small amounts of cryptoassets and interact with the blockchain.    

#Cell 6